In [4]:
#! python3
import praw
import pandas as pd
import datetime as dt
import numpy as np
import matplotlib
matplotlib.rcParams['font.family'] = 'serif'
import matplotlib.pyplot as plt

reddit = praw.Reddit(client_id='biHuZNftcyXJNw',
                     client_secret='NBRuoUNvSSuRp60IqlKZon7l474',
                     user_agent='Learning Curve',
                     username='UltimateFox69',
                     password='ABCD1234')

subreddit = reddit.subreddit('india')

topics_dict = { "title":[], \
                "score":[], \
                "flair":[], \
                "url":[], \
                "num_comments":[], \
                "created": [], \
                "body":[]}

for submission in subreddit.top("year",limit=10000):
    topics_dict["title"].append(submission.title)
    topics_dict["score"].append(submission.score)
    topics_dict["flair"].append(submission.link_flair_text)
    topics_dict["num_comments"].append(submission.num_comments)
    topics_dict["url"].append(submission.url)
    topics_dict["created"].append(submission.created)
    topics_dict["body"].append(submission.selftext)

topics_data = pd.DataFrame(topics_dict)

def get_date(created):
    return dt.datetime.fromtimestamp(created)

_timestamp = topics_data["created"].apply(get_date)
topics_data = topics_data.assign(timestamp = _timestamp)

flair_list = topics_dict["flair"]
commentsNum = topics_dict["num_comments"]
upvotes_num = topics_dict["score"]
flair_dict = {}
flair_comments_per_post = {}
flair_upvotes_per_post = {}
for x in range(len(flair_list)):

  if commentsNum[x] == None:
    commentsNum[x] = 0
  if upvotes_num[x] is None:
    upvotes_num[x] = 0
  if flair_list[x] == None:
    flair_list[x] = "None"
  if flair_list[x] not in flair_dict:
    flair_dict[flair_list[x]] = [commentsNum[x],1,upvotes_num[x],0,0]
  else:
    flair_dict[flair_list[x]][1] += 1
    flair_dict[flair_list[x]][0] += int(commentsNum[x])
  flair_dict[flair_list[x]][3] = flair_dict[flair_list[x]][0]/flair_dict[flair_list[x]][1]
  flair_dict[flair_list[x]][4] = flair_dict[flair_list[x]][2]/flair_dict[flair_list[x]][1]
  flair_comments_per_post[flair_list[x]] = flair_dict[flair_list[x]][3]
  flair_upvotes_per_post[flair_list[x]] = flair_dict[flair_list[x]][4]

# print(i)
# print(flair_dict)
np_flair = np.array(list(flair_comments_per_post.keys()))
np_comments_per_post = np.array(list(flair_comments_per_post.values()))
np_upvotes_per_post = np.array(list(flair_upvotes_per_post.values()))
# print(np_flair)
# print(np_comments_per_post)

def plot_bar_x(y_axis_val, y_label, title):
    # this is for plotting purpose
    index = np.arange(len(np_flair))
    plt.bar(index, y_axis_val)
    plt.xlabel('Flairs', fontsize=15)
    plt.ylabel(y_label, fontsize=15)
    plt.xticks(index, np_flair, fontsize=10, rotation=90)
    plt.title(title)
    plt.show()
    plt.close()
plot_bar_x(np_comments_per_post, 'Comments per Post', 'Comments per Post for each Flair')
plot_bar_x(np_upvotes_per_post, "Upvotes per Post", "Upvotes per Post for each Flair")
#topics_data.to_csv("Midas_data.csv", index=False)

SyntaxError: ignored